In [1]:
import os

import numpy as np
import cv2
import re  # for string splitting
import h5py

from scipy import signal
from droplets import do_cutouts
from droplets import do_metrics
from ipywidgets import interact, interactive, fixed, interact_manual

import matplotlib.pyplot as plt
import ipywidgets as widgets
import numpy.random as random
import scipy.stats as stats
import matplotlib.cm as cm
import matplotlib.patches as patches

from scipy.interpolate import griddata
from scipy.interpolate import interp2d
from scipy.interpolate import RegularGridInterpolator


import gc
%matplotlib

Using matplotlib backend: MacOSX


In [2]:
# directories_with_inputs=['/Users/olofjonsson/Projects/Ice/Droplet_codes/Package/Example_images']#,
#dir_for_results = '/Volumes/LaCie Drive/Lab/Results'

directories_with_inputs = ['/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/45mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/50mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/2.5mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/15mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/20mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/25mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/30mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/35mm',
                           '/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/40mm']


#directories_with_inputs = ['/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/200602/Diffusor_30_40mm_gain40']


dir_for_results = '/Volumes/LaCie Drive/Lab/Results/201104_all'

In [3]:
cuts = []
metrics = []
fig = None
cut = None

run_cutout = False
show_diagnostics = False


for directory_with_input_index, directory_with_input in enumerate(directories_with_inputs):
    path_to_files = directory_with_input
    path_to_results = '/'.join([dir_for_results,
                                'Cuts/' + directory_with_input.split(sep='/')[-1]])
    os.makedirs(path_to_results, exist_ok=True)
    if show_diagnostics:
        path_to_diagnostics = '/'.join([dir_for_results,
                                        'Diagnostics' + directory_with_input])
        os.makedirs(path_to_diagnostics, exist_ok=True)

    list_of_files = [my_file for my_file in os.listdir(
        path_to_files) if not my_file.startswith('.')]  # to avoid hidden files like .DS_file
    print(directory_with_input+':')
    print('found {0} files'.format(str(len(list_of_files))))
    list_of_files.sort()
    print('sorted')

    list_of_files_to_do = list_of_files.copy()[:]

    metrics_list = [None]*len(list_of_files_to_do)
    index_list = [None]*len(list_of_files_to_do)

    if run_cutout:  # Do all

        for file_index, file_name in enumerate(list_of_files_to_do):
            full_path = os.path.join(path_to_files, file_name)
            #image = cv2.imread(full_path)

            print(full_path)

            #temp_file = ImageFile(path_to_files, file_name)
            # metric,cut=GD_metric(image,d_um=2.2)

            cuts, metrics, fig = do_cutouts(full_path, d_um=2.2, smallest_size_um2=10000,
                                            closing_kernel=30, threshold_fraction=0.15, show_diagnostics=show_diagnostics)

            # Make this True if you want to see the cuts
            if False:
                for cut_index, cut in enumerate(cuts):
                    plt.figure()
                    plt.imshow(cut, cmap='gray', vmin=0, vmax=255)
                    # plt.colorbar()
                    plt.title('{0:.0f}'.format(metrics[cut_index]))

            if True:
                for cut_index, cut in enumerate(cuts):
                    cv2.imwrite('{0}/{3}_cut{2}_metric_{1:.0f}.png'.format(path_to_results,
                                                                           metrics[cut_index], cut_index, file_name), cut)

            if show_diagnostics:
                fig.savefig(
                    '{0}/diag_{1}'.format(path_to_diagnostics, file_name), format='png')

            plt.close('all')

    if False:  # Only metrics

        for file_index, file_name in enumerate(list_of_files_to_do):
            full_path = os.path.join(path_to_files, file_name)
            #image = cv2.imread(full_path)

            print(full_path)

            #temp_file = ImageFile(path_to_files, file_name)
            # metric,cut=GD_metric(image,d_um=2.2)
            metrics = do_metrics(full_path, d_um=2.2, smallest_size_um2=10000,
                                 closing_kernel=30, threshold_fraction=0.15)
            metrics_list[file_index] = metrics
            index_list[file_index] = file_index

            plt.close('all')
        # np.save('{0}/metrics_list_0-100'.format(path_to_diagnostics,
        #                                        file_name), metrics_list)
        # np.save(
        #    '{0}/index_list_first0-100'.format(path_to_diagnostics, file_name), index_list)

        #print('{0:.3f}\t {1}'.format(metric,file_name))

/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/45mm:
found 4196 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/50mm:
found 3000 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/2.5mm:
found 21 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/15mm:
found 20 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/20mm:
found 20 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/25mm:
found 100 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/30mm:
found 1100 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/35mm:
found 2000 files
sorted
/Volumes/LaCie Drive/Lab/Raw_data/Home_lab/201104/after_fika/40mm:
found 2000 files
sorted


In [4]:
directories_with_inputs = ['/Volumes/LaCie Drive/Lab/Results/201104/Cuts/35mm', '/Volumes/LaCie Drive/Lab/Results/201104/Cuts/40mm',
                           '/Volumes/LaCie Drive/Lab/Results/201104/Cuts/45mm', '/Volumes/LaCie Drive/Lab/Results/201104/Cuts/50mm']
distances = ['35mm', '40mm', '45mm', '50mm']
colors = ['orange', 'blue', 'green', 'red']

droplet_metrics_all = []

for directory_with_input_index, directory_with_input in enumerate(directories_with_inputs):
    list_of_files = [my_file for my_file in os.listdir(
        directory_with_input) if os.path.isfile(os.path.join(directory_with_input, my_file)) and not my_file.startswith('.')]  # (os.path.isfile(my_file) and (not my_file.startswith('.')))]  # to avoid hidden files like .DS_file
    print(directory_with_input+':')
    print('found {0} files'.format(str(len(list_of_files))))
    list_of_files.sort()
    print('sorted')

    list_of_files_to_do = list_of_files.copy()  # [0:4000:300]
    droplets_metrics = []
    # print(list_of_files_to_do)
    for file_index, file_name in enumerate(list_of_files_to_do):
        splitted = re.split('_', file_name)
        splitted1 = (splitted[-1].split(sep='.')[-2])
        #splitted2 = re.split('.', splitted[-1])
        # print(splitted2)
        droplets_metrics.append(float(splitted1))
    droplet_metrics_all.append(droplets_metrics)

    # print(file_name)
# droplet_metrics_all=np.array(droplet_metrics_all)
# plt.figure()
# plt.hist(droplets_metrics)
# plt.show()

/Volumes/LaCie Drive/Lab/Results/201104/Cuts/35mm:
found 4846 files
sorted
/Volumes/LaCie Drive/Lab/Results/201104/Cuts/40mm:
found 4977 files
sorted
/Volumes/LaCie Drive/Lab/Results/201104/Cuts/45mm:
found 7582 files
sorted
/Volumes/LaCie Drive/Lab/Results/201104/Cuts/50mm:
found 3520 files
sorted


In [5]:
def moving_average(x, w):
    return np.convolve(x, np.ones(w), 'valid') / w


plt.figure('apa')
ax = plt.gca()
plt.figure('korv')
ax2 = plt.gca()

num = 100

bins = np.linspace(0, 5e5, num=num)

# bins=np.append(bins,[[1e6]])

for i, distance in enumerate(distances):
    a, b, c = ax.hist(droplet_metrics_all[i], label=distance, bins=bins,
                      color=colors[i], alpha=1, density=False, histtype='step')
    ax2.plot(moving_average(b, 2), a, label=distance, color=colors[i])
ax.legend()
ax2.legend()

In [6]:
5e5/200

2500.0

In [7]:
a

array([564., 341., 202., 160., 149., 126., 114., 104.,  73.,  82.,  92.,
        69.,  67.,  58.,  64.,  43.,  44.,  49.,  25.,  35.,  38.,  36.,
        35.,  29.,  35.,  19.,  21.,  23.,  32.,  22.,  26.,   9.,  20.,
        15.,  16.,  20.,  18.,  15.,  14.,  15.,  23.,  23.,  16.,  20.,
        15.,  17.,  16.,  14.,  16.,  19.,  17.,  19.,  14.,  16.,  17.,
        26.,  16.,  12.,  15.,  16.,  17.,  18.,  17.,  22.,  24.,  15.,
        19.,  13.,  10.,  13.,  11.,   7.,   7.,   8.,   5.,   3.,   5.,
         2.,   6.,   2.,   1.,   2.,   0.,   1.,   2.,   2.,   2.,   3.,
         1.,   1.,   1.,   0.,   1.,   0.,   2.,   0.,   1.,   0.,   1.])

### STATISTICS BELOW

In [41]:
def q(q):
    '''
    input q value, return q with errors

    q is flow rate
    '''
    return q


def d(d, sigma=0):
    '''
    input d value, return d with errors

    d is droplet diameter
    '''
    d = np.random.normal(loc=d, scale=sigma)
    return d


def s(s, sigma=0):
    '''
    input s value, return s with errors

    s is droplet spacing
    '''
    s = np.random.normal(loc=s, scale=sigma)
    return s


def f(f, count):
    '''
    input f value, return f with errors

    f is fraction
    '''
    f = np.divide(np.random.binomial(count, f), count)
    return f


def l(l):
    '''
    input l value, return l with errors

    l is droplet distance from nozzle
    '''
    return l


def T(l, d, q, s, T_function):
    '''
    input is observables, output is temperature. 

    please use 

    l in m
    d in m
    q in microliters/s
    s in m
    '''

    d_um = d*1e6
    # l_mm=l*1e3
    q_m3_per_s = q*1.666666e-11
    # s_um=s*1e6

    #print(str(l) + ' m distance')
    #print(str(l_mm) + ' mm distance')
    #print(str(d) + ' m diameter')
    #print(str(d_um) + ' um diameter')
    #print(str(s) + ' m spacing')
    #print(str(s_um) + ' um spacing')
    #print(str(q_m3_per_s) + ' flow rate in m^3/s')

    t = np.divide(l*np.pi*d*d*d, 6*q_m3_per_s*s)
    #print(str(t) + ' time in s')

    # v=np.divide(d,t)
    #print(str(v) + ' velocity in m/s')

    # t_ms=t*1e3

    #print(str(t_ms) + ' time in ms')
    T = T_at_time(t, d_um, T_function)

    return T


def J(q, s, d, l_1, l_2, f_1, f_2):
    '''
    input is observables, output is J in unit of 1/(s m^3). 

    please use 

    q in microliters/s
    s in m
    d in m
    l in m
    f as fraction

    we will calculate at length l_1 with fraction f_2
    '''

    q_m3_per_s = q*1.666666e-11

    the_log = np.log(np.divide(np.subtract(1, f_2), np.subtract(1, f_1)))
    factor = np.divide((-36*np.multiply(q_m3_per_s, s)),
                       (np.multiply(np.power(np.pi, 2), np.power(d, 6))))

    J = np.divide(np.multiply(factor, the_log), np.subtract(l_2, l_1))

    return J


def binomial_interval(n, p, method='Wald'):
    z = 1.96
    if method == 'Wald':
        error = z*np.sqrt((1.0-p)*p/n)
    elif method == 'Agresti–Coull':
        error = z*np.sqrt((1.0-p)*p/(n+z*z))
    elif method == 'Wilson_upper':
        z = 1.96
        num = p+z*z/(2*n)+z*np.sqrt(p*(1-p)/n+z*z/(4*n*n))
        den = 1+(z*z)/n
        error = np.divide(num, den)
    elif method == 'Wilson_lower':
        z = -1.96
        num = p+z*z/(2*n)+z*np.sqrt(p*(1-p)/n+z*z/(4*n*n))
        den = 1+(z*z)/n
        error = np.divide(num, den)
    else:
        print('Not an implemented method specified')
    return error


def count_error(measured_fraction, counts):
    '''
    inputs: measured_fraction,counts

    returns: a simulated new fraction using Wilson intervals, and non-symmteric gaussians for the error
    '''
    upper_bound = binomial_interval(
        counts, measured_fraction, method='Wilson_upper')
    lower_bound = binomial_interval(
        counts, measured_fraction, method='Wilson_lower')

    return [lower_bound,upper_bound]

In [9]:
def read_hdf5(path, filename):
    f = h5py.File('{0}/{1}'.format(path, filename), 'r')
    return f['data']['time'], f['data']['Tave']


def T_at_distance(distance, velocity, droplet_size, T_function):
    '''Distance in m, velocity in m/s, droplet_size is diameter in micrometer'''
    time = np.divide(distance, velocity)
    temperature = T_function([droplet_size, time], method='linear')
    return temperature


def T_at_time(time, droplet_size, T_function):
    '''Time in s, droplet_size is diameter in micrometer'''
    temperature = T_function([droplet_size, time], method='linear')
    return temperature


# These are the sizes that are in the folder of hdf5 files. Must have the same timesteps
sizes = [3, 5, 10, 12, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27]

# sizes is the x
x = sizes

# y is the timesteps, and this is taken from the file t_calib_22.h5
time, _ = read_hdf5(
    '/Users/olofjonsson/Projects/Ice/Analysis/From_Niloo/Temp_sim_Jan2010', 't_calib_22.h5')
y = list(time[:])

# Create an empy matrix with the size of the sizes and times
temperatures = np.zeros([len(x), len(y)])

# Filling the matrix with actual values measured.
for size_index, size_value in enumerate(sizes):
    _, temperatures[size_index] = read_hdf5(
        '/Users/olofjonsson/Projects/Ice/Analysis/From_Niloo/Temp_sim_Jan2010', 't_calib_{0}.h5'.format(size_value))

# This is a function returned by the RegularGridInterpolator
#interpolate_T = interp2d(xx, yy, temperatures, kind='linear',bounds_error=True)
interpolate_T = RegularGridInterpolator((x, y), temperatures)

In [10]:
counts = np.multiply([48, 27, 34, 210, 715, 881, 797, 292], 1)
fractions = [0, 0.03, 0.00, 0.06, 0.04, 0.04,
             0.13, 0.17]  # ice fraction for focused
distances = [15, 20, 25, 30, 35, 40, 45, 50]

# relabelling
l_array = np.multiply(distances, 1e-3)
f_array = fractions


d_sigma = 0.496e-6
d_value = 20.2e-6

s_value = 58.3e-6
s_sigma = 3.67e-6

q_value = 56.5

trials = range(1000)

In [11]:
# fig,axes=plt.subplots(nrows=2,ncols=1,figsize=[10,10])
# fig.canvas.set_window_title('Large')
# ax1=axes[0]#[0,0]
# ax2=axes[1]#[1,0]


'''#With non-fraction errors
for trial in trials:
    d_trial=d(d_value,d_sigma)
    s_trial=s(s_value,s_sigma)
    q_trial=q(q_value)

    l_length=len(l_array)
    
    temperatures_trial=np.zeros(l_length)
    J_trial=np.zeros(len(l_array))
    for l_index,l_value in enumerate(l_array):
        temperatures_trial[l_index] = T(l_value,d_trial,s_trial,q_trial,interpolate_T)
        if (l_index+1)<l_length:
            
            next_l=l_array[l_index+1]
            now_l=l_value
            next_f=f_array[l_index+1]
            now_f=f_array[l_index]
            
            J_trial[l_index]=J(q_trial,s_trial,d_trial,next_l,l_value,next_f,now_f)
        else:
            J_trial[l_index]=np.nan
            
    #ax1.scatter(temperatures_trial,J_trial,c='black',marker='o',alpha=0.05)'''

'''#With only fractions
for trial in trials:
    d_trial=d(d_value,0)
    s_trial=s(s_value,0)
    q_trial=q(q_value)

    l_length=len(l_array)
    
    temperatures_trial=np.zeros(l_length)
    J_trial=np.zeros(len(l_array))
    
    f_with_errors=np.zeros(len(l_array))
    
    for f_index,f_value in enumerate(f_array):
        f_with_errors[f_index]=f(f_value,counts[f_index])
    
    for l_index,l_value in enumerate(l_array):
        temperatures_trial[l_index] = T(l_value,d_trial,s_trial,q_trial,interpolate_T)
        if (l_index+1)<l_length:
            
            next_l=l_array[l_index+1]
            now_l=l_value
            next_f=f_with_errors[l_index+1]
            now_f=f_with_errors[l_index]
            
            J_trial[l_index]=J(q_trial,s_trial,d_trial,next_l,l_value,next_f,now_f)
        else:
            J_trial[l_index]=np.nan
            
    #ax1.scatter(temperatures_trial,J_trial,c='green',marker='o',alpha=0.05)'''


T_for_hist = []
J_for_hist = []

# With all errors
for trial in trials:
    d_trial = d(d_value, d_sigma)
    s_trial = s(s_value, s_sigma)
    q_trial = q(q_value)

    l_length = len(l_array)

    temperatures_trial = np.zeros(l_length)
    J_trial = np.zeros(len(l_array))

    f_with_errors = np.zeros(len(l_array))

    for f_index, f_value in enumerate(f_array):
        f_with_errors[f_index] = f(f_value, counts[f_index])

    for l_index, l_value in enumerate(l_array):
        temperatures_trial[l_index] = T(
            l_value, d_trial, s_trial, q_trial, interpolate_T)
        if (l_index+1) < l_length:

            next_l = l_array[l_index+1]
            now_l = l_value
            next_f = f_with_errors[l_index+1]
            now_f = f_with_errors[l_index]

            J_trial[l_index] = J(q_trial, s_trial, d_trial,
                                 next_l, l_value, next_f, now_f)
        else:
            J_trial[l_index] = np.nan

        T_for_hist.append(temperatures_trial[l_index])
        J_for_hist.append(J_trial[l_index])

    # ax1.scatter(temperatures_trial,J_trial,c='blue',marker='o',alpha=0.05)

J_for_hist_err, T_for_hist_err = J_for_hist, T_for_hist

# IDEAL
d_trial = d(d_value, 0)
s_trial = s(s_value, 0)
q_trial = q(q_value)

l_length = len(l_array)

temperatures_trial = np.zeros(l_length)
J_trial = np.zeros(len(l_array))

del next_f
del now_f

for l_index, l_value in enumerate(l_array):
    temperatures_trial[l_index] = T(
        l_value, d_trial, s_trial, q_trial, interpolate_T)
    if (l_index+1) < l_length:

        next_l = l_array[l_index+1]
        now_l = l_value
        next_f = f_array[l_index+1]
        now_f = f_array[l_index]

        J_trial[l_index] = J(q_trial, s_trial, d_trial,
                             next_l, l_value, next_f, now_f)
    else:
        J_trial[l_index] = np.nan

T_ideal, J_ideal = temperatures_trial, J_trial
'''
ax1.plot(temperatures_trial,J_trial,c='red',marker='o')

ax1.set_yscale('symlog',linthreshy=1e16,linscaley=0.2)
ax1.set_ylabel('J in 1/(s m^3)')
ax1.set_xlabel('T in K')
'''

"\nax1.plot(temperatures_trial,J_trial,c='red',marker='o')\n\nax1.set_yscale('symlog',linthreshy=1e16,linscaley=0.2)\nax1.set_ylabel('J in 1/(s m^3)')\nax1.set_xlabel('T in K')\n"

In [12]:
plt.close()
plt.figure('Small', figsize=[5, 5])

ax2 = plt.gca()


# ax2.plot(temperatures_trial,J_trial,c='red',marker='o')
ax2.set_ylabel('J in 1/(s m^3)')
ax2.set_xlabel('T in K')
xmin = 227  # np.min(T_for_hist)
xmax = 250  # np.max(T_for_hist)
ymin = -1e17
ymax = 5e18
ax2.set_yscale('symlog',linthresh=5e15,linscale=0.1,subs=[2, 3, 4, 5, 6, 7, 8, 9])
ax2.set_xlim([xmin,xmax])
ax2.set_ylim([ymin,ymax])
#counts_in_hist,xbins,ybins,histogram=ax2.hist2d(T_for_hist,J_for_hist,range=[[xmin,xmax], [ymin, ymax]],bins=200,cmin = 1,cmap='Reds')
# ax2.contour(counts_in_hist,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3)
counts_in_hist, xbins, ybins = np.histogram2d(J_for_hist_err, T_for_hist_err, range=[
                                              [ymin, ymax], [xmin, xmax]], density=False, bins=50)

####histogram = ax2.pcolormesh(ybins, xbins, counts_in_hist, cmap='Reds')
# ax2.imshow(counts_in_hist)

x_bin_centers = [np.average([xbins[i+1], xbins[i]])
                 for i, value in enumerate(xbins[:-1])]
y_bin_centers = [np.average([ybins[i+1], ybins[i]])
                 for i, value in enumerate(ybins[:-1])]
X, Y = np.meshgrid(x_bin_centers, y_bin_centers)

x_low, x_high = ax2.get_xlim()
y_low, y_high = ax2.get_ylim()
extents = (x_low, x_high, y_low, y_high)

# contour_ax2=ax2.contour(Y,X,counts_in_hist,extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],linewidths=3)
# contour_ax2=ax2.contour(Y,X,counts_in_hist,extent=extents,linewidths=1)

# levels=[0.1,0.2,0.5,5,10]
# levels=[5,10]


# contour_data=np.divide(counts_in_hist.T,np.max(counts_in_hist))
contour_data = counts_in_hist.T

levels = np.linspace(0, np.max(contour_data), num=10)[1:]

# contour_ax2=ax2.contour(Y,X,contour_data,linewidths=1,levels=levels,colors='black')

ax2.plot(T_ideal, J_ideal, c='red', marker='o', label='Spring')
ax2.legend()
# ax2.set_yscale('symlog',linthreshy=4e16,linscaley=0.2)
# ax2.set_yscale('log')
#######cbar = plt.colorbar(histogram, ax=ax2)
# cbar.add_lines(contour_ax2)
# plt.savefig('countour_lin_better_microscopy_errors')


percentile = 5

T_low = np.zeros(len(distances))
T_high = np.zeros(len(distances))
J_low = np.zeros(len(distances))
J_high = np.zeros(len(distances))

values = range(len(counts))

for value_i, value in enumerate(values):
    cycle = len(counts)

    T_err = np.roll(np.array(T_for_hist), -value)[::cycle]
    J_err = np.roll(np.array(J_for_hist), -value)[::cycle]

    T_low[value_i] = T_ideal[value_i]-np.percentile(T_err, percentile)
    T_high[value_i] = np.percentile(T_err, 100-percentile)-T_ideal[value_i]

    J_low[value_i] = J_ideal[value_i]-np.percentile(J_err, percentile)
    J_high[value_i] = np.percentile(J_err, 100-percentile)-J_ideal[value_i]

# plt.figure('errors')
# ax=plt.gca()
xerr = np.array([T_low, T_high])
yerr = np.array([J_low, J_high])
ax2.errorbar(T_ideal, J_ideal, xerr=xerr, yerr=yerr,
             c='red', fmt='none', capsize=5)

<ErrorbarContainer object of 3 artists>

In [13]:
##############NOVEMBER EXPERIMENT


#############   TESTING

plt.close()


counts = np.multiply([48, 27, 34, 210, 715, 881, 797, 292], 1)
fractions = [0, 0.03, 0.00, 0.06, 0.04, 0.04,
             0.13, 0.17]  # ice fraction for focused
distances = [15, 20, 25, 30, 35, 40, 45, 50]

counts = np.multiply([48, 27, 34, 210, 715, 881, 797, 84], 1)
fractions = [0, 0.03, 0.00, 0.06, 0.04, 0.04,
             0.13, 0.35]  # ice fraction for focused
distances = [15, 20, 25, 30, 35, 40, 45, 50]


# relabelling
l_array = np.multiply(distances, 1e-3)
f_array = fractions


d_sigma = 0.496e-6
d_value = 20.2e-6

s_value = 58.3e-6
s_sigma = 3.67e-6

q_value = 56.5

trials = range(1000)
#percentile = 15.87 #One sigma
percentile = 5 #Two sigma


#plt.close()
plt.figure('Small', figsize=[5, 5])

ax2 = plt.gca()


# ax2.plot(temperatures_trial,J_trial,c='red',marker='o')
ax2.set_ylabel('J in 1/(s m^3)')
ax2.set_xlabel('T in K')
xmin = 225  # np.min(T_for_hist)
xmax = 250  # np.max(T_for_hist)
#ymax = 5e18
ymax=5e18


log=False
if log:
    ymin=1e16
    ax2.set_yscale('log')
else:
    ymin=0
    ymax=2.3e18
    
    
ax2.grid(which='both',alpha=0.4)
ax2.set_xlim([xmin,xmax])
ax2.set_ylim([ymin,ymax])

ax2.plot(T_ideal, J_ideal, c='blue', marker='o', label='3 Nov')
ax2.legend()


xerr = np.array([T_low, T_high])
yerr = np.array([J_low, J_high])
ax2.errorbar(T_ideal, J_ideal, xerr=xerr, yerr=yerr,
             c='blue', fmt='none', capsize=5)

<ErrorbarContainer object of 3 artists>

In [99]:
##############NOVEMBER EXPERIMENT


plt.close()


counts = np.multiply([48, 27, 34, 210, 715, 881, 797, 292], 1)
fractions = [0, 0.03, 0.00, 0.06, 0.04, 0.04,
             0.13, 0.17]  # ice fraction for focused
distances = [15, 20, 25, 30, 35, 40, 45, 50]

counts = np.multiply([48, 27, 34, 210, 715, 881, 797, 84], 1)
fractions = [0, 0.03, 0.00, 0.06, 0.04, 0.04,
             0.13, 0.35]  # ice fraction for focused
distances = [15, 20, 25, 30, 35, 40, 45, 50]


# relabelling
l_array = np.multiply(distances, 1e-3)
f_array = fractions


d_sigma = 0.496e-6
d_value = 20.2e-6

s_value = 58.3e-6
s_sigma = 3.67e-6

q_value = 56.5

trials = range(1000)
#percentile = 15.87 #One sigma
percentile = 5 #Two sigma


# fig,axes=plt.subplots(nrows=2,ncols=1,figsize=[10,10])
# fig.canvas.set_window_title('Large')
# ax1=axes[0]#[0,0]
# ax2=axes[1]#[1,0]


'''#With non-fraction errors
for trial in trials:
    d_trial=d(d_value,d_sigma)
    s_trial=s(s_value,s_sigma)
    q_trial=q(q_value)

    l_length=len(l_array)
    
    temperatures_trial=np.zeros(l_length)
    J_trial=np.zeros(len(l_array))
    for l_index,l_value in enumerate(l_array):
        temperatures_trial[l_index] = T(l_value,d_trial,s_trial,q_trial,interpolate_T)
        if (l_index+1)<l_length:
            
            next_l=l_array[l_index+1]
            now_l=l_value
            next_f=f_array[l_index+1]
            now_f=f_array[l_index]
            
            J_trial[l_index]=J(q_trial,s_trial,d_trial,next_l,l_value,next_f,now_f)
        else:
            J_trial[l_index]=np.nan
            
    #ax1.scatter(temperatures_trial,J_trial,c='black',marker='o',alpha=0.05)'''

'''#With only fractions
for trial in trials:
    d_trial=d(d_value,0)
    s_trial=s(s_value,0)
    q_trial=q(q_value)

    l_length=len(l_array)
    
    temperatures_trial=np.zeros(l_length)
    J_trial=np.zeros(len(l_array))
    
    f_with_errors=np.zeros(len(l_array))
    
    for f_index,f_value in enumerate(f_array):
        f_with_errors[f_index]=f(f_value,counts[f_index])
    
    for l_index,l_value in enumerate(l_array):
        temperatures_trial[l_index] = T(l_value,d_trial,s_trial,q_trial,interpolate_T)
        if (l_index+1)<l_length:
            
            next_l=l_array[l_index+1]
            now_l=l_value
            next_f=f_with_errors[l_index+1]
            now_f=f_with_errors[l_index]
            
            J_trial[l_index]=J(q_trial,s_trial,d_trial,next_l,l_value,next_f,now_f)
        else:
            J_trial[l_index]=np.nan
            
    #ax1.scatter(temperatures_trial,J_trial,c='green',marker='o',alpha=0.05)'''


T_for_hist = []
J_for_hist = []

# With all errors
for trial in trials:
    d_trial = d(d_value, d_sigma)
    s_trial = s(s_value, s_sigma)
    q_trial = q(q_value)

    l_length = len(l_array)

    temperatures_trial = np.zeros(l_length)
    J_trial = np.zeros(len(l_array))

    f_with_errors = np.zeros(len(l_array))

    for f_index, f_value in enumerate(f_array):
        f_with_errors[f_index] = f(f_value, counts[f_index])

    for l_index, l_value in enumerate(l_array):
        if (l_index+1) < l_length:
            
           

            next_l = l_array[l_index+1]
            now_l = l_value
            next_f = f_with_errors[l_index+1]
            now_f = f_with_errors[l_index]
            
            temperatures_trial[l_index] = T(np.average([next_l,now_l]), d_trial, s_trial, q_trial, interpolate_T)

            J_trial[l_index] = J(q_trial, s_trial, d_trial,
                                 next_l, l_value, next_f, now_f)
        else:
            J_trial[l_index] = np.nan
            temperatures_trial[l_index]= np.nan

        T_for_hist.append(temperatures_trial[l_index])
        J_for_hist.append(J_trial[l_index])

    # ax1.scatter(temperatures_trial,J_trial,c='blue',marker='o',alpha=0.05)

J_for_hist_err, T_for_hist_err = J_for_hist, T_for_hist

# IDEAL
d_trial = d(d_value, 0)
s_trial = s(s_value, 0)
q_trial = q(q_value)

l_length = len(l_array)

temperatures_trial = np.zeros(l_length)
J_trial = np.zeros(len(l_array))

del next_f
del now_f

for l_index, l_value in enumerate(l_array):

    if (l_index+1) < l_length:

        next_l = l_array[l_index+1]
        now_l = l_value
        next_f = f_array[l_index+1]
        now_f = f_array[l_index]

        temperatures_trial[l_index] = T(np.average([next_l,now_l]), d_trial, s_trial, q_trial, interpolate_T)
        J_trial[l_index] = J(q_trial, s_trial, d_trial,
                             next_l, l_value, next_f, now_f)
    else:
        J_trial[l_index] = np.nan
        temperatures_trial[l_index]=np.nan

T_ideal, J_ideal = temperatures_trial, J_trial
'''
ax1.plot(temperatures_trial,J_trial,c='red',marker='o')

ax1.set_yscale('symlog',linthreshy=1e16,linscaley=0.2)
ax1.set_ylabel('J in 1/(s m^3)')
ax1.set_xlabel('T in K')
'''

#plt.close()
plt.figure('Small', figsize=[5, 5])

ax2 = plt.gca()


# ax2.plot(temperatures_trial,J_trial,c='red',marker='o')
ax2.set_ylabel('J in 1/(s m^3)')
ax2.set_xlabel('T in K')
xmin = 225  # np.min(T_for_hist)
xmax = 250  # np.max(T_for_hist)
#ymax = 5e18
ymax=5e18


log=False
if log:
    ymin=1e16
    ax2.set_yscale('log')
else:
    ymin=0
    ymax=2.3e18
    
    
#ax2.grid(which='both')  
ax2.grid(which='both',alpha=0.4)
#ax2.set_yscale('symlog',linthresh=2e16,linscale=0.1,subs=[2, 3, 4, 5, 6, 7, 8, 9])
ax2.set_xlim([xmin,xmax])
ax2.set_ylim([ymin,ymax])
#counts_in_hist,xbins,ybins,histogram=ax2.hist2d(T_for_hist,J_for_hist,range=[[xmin,xmax], [ymin, ymax]],bins=200,cmin = 1,cmap='Reds')
# ax2.contour(counts_in_hist,extent=[xbins.min(),xbins.max(),ybins.min(),ybins.max()],linewidths=3)
counts_in_hist, xbins, ybins = np.histogram2d(J_for_hist_err, T_for_hist_err, range=[
                                              [ymin, ymax], [xmin, xmax]], density=False, bins=50)

############histogram = ax2.pcolormesh(ybins, xbins, counts_in_hist, cmap='Reds')
# ax2.imshow(counts_in_hist)

x_bin_centers = [np.average([xbins[i+1], xbins[i]])
                 for i, value in enumerate(xbins[:-1])]
y_bin_centers = [np.average([ybins[i+1], ybins[i]])
                 for i, value in enumerate(ybins[:-1])]
X, Y = np.meshgrid(x_bin_centers, y_bin_centers)

x_low, x_high = ax2.get_xlim()
y_low, y_high = ax2.get_ylim()
extents = (x_low, x_high, y_low, y_high)

# contour_ax2=ax2.contour(Y,X,counts_in_hist,extent=[xbins[0],xbins[-1],ybins[0],ybins[-1]],linewidths=3)
# contour_ax2=ax2.contour(Y,X,counts_in_hist,extent=extents,linewidths=1)

# levels=[0.1,0.2,0.5,5,10]
# levels=[5,10]


# contour_data=np.divide(counts_in_hist.T,np.max(counts_in_hist))
contour_data = counts_in_hist.T

levels = np.linspace(0, np.max(contour_data), num=10)[1:]

# contour_ax2=ax2.contour(Y,X,contour_data,linewidths=1,levels=levels,colors='black')

ax2.plot(T_ideal, J_ideal, c='blue', marker='o', label='3 Nov')
ax2.legend()
# ax2.set_yscale('symlog',linthreshy=4e16,linscaley=0.2)
# ax2.set_yscale('log')
#######cbar = plt.colorbar(histogram, ax=ax2)
# cbar.add_lines(contour_ax2)
# plt.savefig('countour_lin_better_microscopy_errors')




T_low = np.zeros(len(distances))
T_high = np.zeros(len(distances))
J_low = np.zeros(len(distances))
J_high = np.zeros(len(distances))

values = range(len(counts))

for value_i, value in enumerate(values):
    cycle = len(counts)

    T_err = np.roll(np.array(T_for_hist), -value)[::cycle]
    J_err = np.roll(np.array(J_for_hist), -value)[::cycle]

    T_low[value_i] = T_ideal[value_i]-np.percentile(T_err, percentile)
    T_high[value_i] = np.percentile(T_err, 100-percentile)-T_ideal[value_i]

    J_low[value_i] = J_ideal[value_i]-np.percentile(J_err, percentile)
    J_high[value_i] = np.percentile(J_err, 100-percentile)-J_ideal[value_i]

# plt.figure('errors')
# ax=plt.gca()
xerr = np.array([T_low, T_high])
yerr = np.array([J_low, J_high])
ax2.errorbar(T_ideal, J_ideal, xerr=xerr, yerr=yerr,
             c='blue', fmt='none', capsize=5)

<ErrorbarContainer object of 3 artists>

In [100]:
print('Nov 3\n')


print('T_ideal\n' + str(T_ideal))
print('J_ideal\n' + str(J_ideal))


print('T_low\n' + str(T_low))
print('T_high\n' + str(T_high))
print('J_low\n' + str(J_low))
print('J_high\n' + str(J_high))



print('\nCount errors\n')

W_errors=np.multiply(np.array([count_error(fractions[i],counts[i]) for i in range(len(fractions))]),1)
print('Distances in mm')
print(distances)
print('Low error (0-1)')
low=np.subtract(fractions,W_errors[:,0])
print(low)
print('High error (0-1)')
high=-np.subtract(fractions,W_errors[:,1])
print(high)


plt.figure('Count_errors')
plt.errorbar(distances,fractions,yerr=[low,high])

Nov 3

T_ideal
[242.0594692  239.17767685 237.00287376 235.27730872 233.85920182
 232.66300914 231.63352692          nan]
J_ideal
[ 1.79559634e+16 -1.79559634e+16  3.64760799e+16 -1.24111649e+16
  0.00000000e+00  5.80312652e+16  1.71854035e+17             nan]
T_low
[1.53828937 1.45160709 1.37982903 1.32110787 1.27205629 1.23043099
 1.19463196        nan]
T_high
[1.54383905 1.46331537 1.39710141 1.34169907 1.29471735 1.25414298
 1.21888026        nan]
J_low
[1.79559634e+16 4.50803336e+16 1.65475224e+16 2.14709590e+16
 9.85124229e+15 1.89573154e+16 8.07969382e+16            nan]
J_high
[4.50803336e+16 1.79559634e+16 2.38743069e+16 1.76534113e+16
 9.45245282e+15 2.46130850e+16 1.10706725e+17            nan]

Count errors

Distances in mm
[15, 20, 25, 30, 35, 40, 45, 50]
Low error (0-1)
[0.         0.02543297 0.         0.02489324 0.01207643 0.01106828
 0.02158504 0.09340176]
High error (0-1)
[0.07410265 0.14251847 0.10151791 0.04070218 0.01699306 0.01506252
 0.02513479 0.10652174]


<ErrorbarContainer object of 3 artists>

In [101]:
### EARLIER EXPERIMENT


counts = np.multiply([113,471,319,406,327,456,206,220],1)
fractions = [0.02,0.06,0.09,0.07,0.07,0.17,0.62,0.92] #ice fraction for focused
distances = [30,35,37.5,40,42.5,45,47.5,50]



# relabelling
l_array = np.multiply(distances, 1e-3)
f_array = fractions


d_sigma=0.95e-6
d_value=21.45e-6
s_sigma=1.4e-6
s_value=58.8e-6
q_value=55

T_for_hist = []
J_for_hist = []

# With all errors
for trial in trials:
    d_trial = d(d_value, d_sigma)
    s_trial = s(s_value, s_sigma)
    q_trial = q(q_value)

    l_length = len(l_array)

    temperatures_trial = np.zeros(l_length)
    J_trial = np.zeros(len(l_array))

    f_with_errors = np.zeros(len(l_array))

    for f_index, f_value in enumerate(f_array):
        f_with_errors[f_index] = f(f_value, counts[f_index])

    for l_index, l_value in enumerate(l_array):
        if (l_index+1) < l_length:
            
           

            next_l = l_array[l_index+1]
            now_l = l_value
            next_f = f_with_errors[l_index+1]
            now_f = f_with_errors[l_index]
            
            temperatures_trial[l_index] = T(np.average([next_l,now_l]), d_trial, s_trial, q_trial, interpolate_T)

            J_trial[l_index] = J(q_trial, s_trial, d_trial,
                                 next_l, l_value, next_f, now_f)
        else:
            J_trial[l_index] = np.nan
            temperatures_trial[l_index]= np.nan

        T_for_hist.append(temperatures_trial[l_index])
        J_for_hist.append(J_trial[l_index])


    # ax1.scatter(temperatures_trial,J_trial,c='blue',marker='o',alpha=0.05)

J_for_hist_err, T_for_hist_err = J_for_hist, T_for_hist

# IDEAL
d_trial = d(d_value, 0)
s_trial = s(s_value, 0)
q_trial = q(q_value)

l_length = len(l_array)

temperatures_trial = np.zeros(l_length)
J_trial = np.zeros(len(l_array))

del next_f
del now_f

for l_index, l_value in enumerate(l_array):

    if (l_index+1) < l_length:

        next_l = l_array[l_index+1]
        now_l = l_value
        next_f = f_array[l_index+1]
        now_f = f_array[l_index]

        temperatures_trial[l_index] = T(np.average([next_l,now_l]), d_trial, s_trial, q_trial, interpolate_T)
        J_trial[l_index] = J(q_trial, s_trial, d_trial,
                             next_l, l_value, next_f, now_f)
    else:
        J_trial[l_index] = np.nan
        temperatures_trial[l_index]=np.nan

T_ideal, J_ideal = temperatures_trial, J_trial

ax2.plot(T_ideal, J_ideal, c='red', marker='o', label='Spring')
ax2.legend()
'''
ax1.plot(temperatures_trial,J_trial,c='red',marker='o')

ax1.set_yscale('symlog',linthreshy=1e16,linscaley=0.2)
ax1.set_ylabel('J in 1/(s m^3)')
ax1.set_xlabel('T in K')
'''

#plt.close()

T_low = np.zeros(len(distances))
T_high = np.zeros(len(distances))
J_low = np.zeros(len(distances))
J_high = np.zeros(len(distances))

values = range(len(counts))

for value_i, value in enumerate(values):
    cycle = len(counts)

    T_err = np.roll(np.array(T_for_hist), -value)[::cycle]
    J_err = np.roll(np.array(J_for_hist), -value)[::cycle]

    T_low[value_i] = T_ideal[value_i]-np.percentile(T_err, percentile)
    T_high[value_i] = np.percentile(T_err, 100-percentile)-T_ideal[value_i]

    J_low[value_i] = J_ideal[value_i]-np.percentile(J_err, percentile)
    J_high[value_i] = np.percentile(J_err, 100-percentile)-J_ideal[value_i]

# plt.figure('errors')
# ax=plt.gca()
xerr = np.array([T_low, T_high])
yerr = np.array([J_low, J_high])
ax2.errorbar(T_ideal, J_ideal, xerr=xerr, yerr=yerr,
             c='red', fmt='none', capsize=5)

<ErrorbarContainer object of 3 artists>

In [102]:
print('Spring')


print('T_ideal\n' + str(T_ideal))
print('J_ideal\n' + str(J_ideal))


print('T_low\n' + str(T_low))
print('T_high\n' + str(T_high))
print('J_low\n' + str(J_low))
print('J_high\n' + str(J_high))

print('\nCount errors\n')

W_errors=np.multiply(np.array([count_error(fractions[i],counts[i]) for i in range(len(fractions))]),1)

print('Distances in mm')
print(distances)
print('Low error (0-1)')
low=np.subtract(fractions,W_errors[:,0])
print(low)
print('High error (0-1)')
high=-np.subtract(fractions,W_errors[:,1])
print(high)


plt.figure('Count_errors')
plt.errorbar(distances,fractions,yerr=[low,high])

Spring
T_ideal
[233.94843493 232.89695972 232.27005986 231.69265282 231.15815199
 230.66115462 230.1971802           nan]
J_ideal
[ 1.68232216e+16  2.61881701e+16 -1.75528173e+16  0.00000000e+00
  9.18486732e+16  6.30783119e+17  1.25804253e+18             nan]
T_low
[1.37549529 1.3380407  1.31560586 1.29491247 1.27575766 1.2579688
 1.24139819        nan]
T_high
[1.60437852 1.56176101 1.5360675  1.51226229 1.49013908 1.46952048
 1.45025184        nan]
J_low
[1.24060483e+16 2.63063253e+16 3.63343498e+16 3.05790466e+16
 4.13242942e+16 2.38396441e+17 5.15445202e+17            nan]
J_high
[1.77331626e+16 3.50048900e+16 3.12403173e+16 3.13769948e+16
 6.95374474e+16 4.06281363e+17 9.22613175e+17            nan]

Count errors

Distances in mm
[30, 35, 37.5, 40, 42.5, 45, 47.5, 50]
Low error (0-1)
[0.01410948 0.01809584 0.02671807 0.02099848 0.02295062 0.03168692
 0.06790826 0.04347227]
High error (0-1)
[0.04567303 0.02521528 0.03647552 0.02905958 0.03293659 0.03720068
 0.06351455 0.02905607]


<ErrorbarContainer object of 3 artists>

In [32]:
###### With higher counts

counts = np.multiply([48, 27, 34, 210, 715, 881, 797, 84], 10)
fractions = [0, 0.03, 0.00, 0.06, 0.04, 0.04,
             0.13, 0.35]  # ice fraction for focused
distances = [15, 20, 25, 30, 35, 40, 45, 50]


# relabelling
l_array = np.multiply(distances, 1e-3)
f_array = fractions


d_sigma = 0.496e-6
d_value = 20.2e-6

s_value = 58.3e-6
s_sigma = 3.67e-6

q_value = 56.5

T_for_hist = []
J_for_hist = []

# With all errors
for trial in trials:
    d_trial = d(d_value, d_sigma)
    s_trial = s(s_value, s_sigma)
    q_trial = q(q_value)

    l_length = len(l_array)

    temperatures_trial = np.zeros(l_length)
    J_trial = np.zeros(len(l_array))

    f_with_errors = np.zeros(len(l_array))

    for f_index, f_value in enumerate(f_array):
        f_with_errors[f_index] = f(f_value, counts[f_index])

    for l_index, l_value in enumerate(l_array):
        temperatures_trial[l_index] = T(
            l_value, d_trial, s_trial, q_trial, interpolate_T)
        if (l_index+1) < l_length:

            next_l = l_array[l_index+1]
            now_l = l_value
            next_f = f_with_errors[l_index+1]
            now_f = f_with_errors[l_index]

            J_trial[l_index] = J(q_trial, s_trial, d_trial,
                                 next_l, l_value, next_f, now_f)
        else:
            J_trial[l_index] = np.nan

        T_for_hist.append(temperatures_trial[l_index])
        J_for_hist.append(J_trial[l_index])

    # ax1.scatter(temperatures_trial,J_trial,c='blue',marker='o',alpha=0.05)

J_for_hist_err, T_for_hist_err = J_for_hist, T_for_hist

# IDEAL
d_trial = d(d_value, 0)
s_trial = s(s_value, 0)
q_trial = q(q_value)

l_length = len(l_array)

temperatures_trial = np.zeros(l_length)
J_trial = np.zeros(len(l_array))

del next_f
del now_f

for l_index, l_value in enumerate(l_array):
    temperatures_trial[l_index] = T(
        l_value, d_trial, s_trial, q_trial, interpolate_T)
    if (l_index+1) < l_length:

        next_l = l_array[l_index+1]
        now_l = l_value
        next_f = f_array[l_index+1]
        now_f = f_array[l_index]

        J_trial[l_index] = J(q_trial, s_trial, d_trial,
                             next_l, l_value, next_f, now_f)
    else:
        J_trial[l_index] = np.nan

T_ideal, J_ideal = temperatures_trial, J_trial

ax2.plot(T_ideal, J_ideal, c='red', marker='o', label='3 nov, better count')
ax2.legend()
'''
ax1.plot(temperatures_trial,J_trial,c='red',marker='o')

ax1.set_yscale('symlog',linthreshy=1e16,linscaley=0.2)
ax1.set_ylabel('J in 1/(s m^3)')
ax1.set_xlabel('T in K')
'''

#plt.close()

T_low = np.zeros(len(distances))
T_high = np.zeros(len(distances))
J_low = np.zeros(len(distances))
J_high = np.zeros(len(distances))

values = range(len(counts))

for value_i, value in enumerate(values):
    cycle = len(counts)

    T_err = np.roll(np.array(T_for_hist), -value)[::cycle]
    J_err = np.roll(np.array(J_for_hist), -value)[::cycle]

    T_low[value_i] = T_ideal[value_i]-np.percentile(T_err, percentile)
    T_high[value_i] = np.percentile(T_err, 100-percentile)-T_ideal[value_i]

    J_low[value_i] = J_ideal[value_i]-np.percentile(J_err, percentile)
    J_high[value_i] = np.percentile(J_err, 100-percentile)-J_ideal[value_i]

# plt.figure('errors')
# ax=plt.gca()
xerr = np.array([T_low, T_high])
yerr = np.array([J_low, J_high])
ax2.errorbar(T_ideal, J_ideal, xerr=xerr, yerr=yerr,
             c='red', fmt='none', capsize=5)

<ErrorbarContainer object of 3 artists>

In [61]:
###### With better spacing

counts = np.multiply([48, 27, 34, 210, 715, 881, 797, 84], 1)
fractions = [0, 0.03, 0.00, 0.06, 0.04, 0.04,
             0.13, 0.35]  # ice fraction for focused
distances = [15, 20, 25, 30, 35, 40, 45, 50]


# relabelling
l_array = np.multiply(distances, 1e-3)
f_array = fractions


d_sigma = 0.496e-6
d_value = 20.2e-6

s_value = 58.3e-6
s_sigma = 3.67e-6/4

q_value = 56.5

T_for_hist = []
J_for_hist = []

# With all errors
for trial in trials:
    d_trial = d(d_value, d_sigma)
    s_trial = s(s_value, s_sigma)
    q_trial = q(q_value)

    l_length = len(l_array)

    temperatures_trial = np.zeros(l_length)
    J_trial = np.zeros(len(l_array))

    f_with_errors = np.zeros(len(l_array))

    for f_index, f_value in enumerate(f_array):
        f_with_errors[f_index] = f(f_value, counts[f_index])

    for l_index, l_value in enumerate(l_array):
        temperatures_trial[l_index] = T(
            l_value, d_trial, s_trial, q_trial, interpolate_T)
        if (l_index+1) < l_length:

            next_l = l_array[l_index+1]
            now_l = l_value
            next_f = f_with_errors[l_index+1]
            now_f = f_with_errors[l_index]

            J_trial[l_index] = J(q_trial, s_trial, d_trial,
                                 next_l, l_value, next_f, now_f)
        else:
            J_trial[l_index] = np.nan

        T_for_hist.append(temperatures_trial[l_index])
        J_for_hist.append(J_trial[l_index])

    # ax1.scatter(temperatures_trial,J_trial,c='blue',marker='o',alpha=0.05)

J_for_hist_err, T_for_hist_err = J_for_hist, T_for_hist

# IDEAL
d_trial = d(d_value, 0)
s_trial = s(s_value, 0)
q_trial = q(q_value)

l_length = len(l_array)

temperatures_trial = np.zeros(l_length)
J_trial = np.zeros(len(l_array))

del next_f
del now_f

for l_index, l_value in enumerate(l_array):
    temperatures_trial[l_index] = T(
        l_value, d_trial, s_trial, q_trial, interpolate_T)
    if (l_index+1) < l_length:

        next_l = l_array[l_index+1]
        now_l = l_value
        next_f = f_array[l_index+1]
        now_f = f_array[l_index]

        J_trial[l_index] = J(q_trial, s_trial, d_trial,
                             next_l, l_value, next_f, now_f)
    else:
        J_trial[l_index] = np.nan

T_ideal, J_ideal = temperatures_trial, J_trial

ax2.plot(T_ideal, J_ideal, c='green', marker='o', label='3 nov, better spacing')
ax2.legend()
'''
ax1.plot(temperatures_trial,J_trial,c='red',marker='o')

ax1.set_yscale('symlog',linthreshy=1e16,linscaley=0.2)
ax1.set_ylabel('J in 1/(s m^3)')
ax1.set_xlabel('T in K')
'''

#plt.close()

T_low = np.zeros(len(distances))
T_high = np.zeros(len(distances))
J_low = np.zeros(len(distances))
J_high = np.zeros(len(distances))

values = range(len(counts))

for value_i, value in enumerate(values):
    cycle = len(counts)

    T_err = np.roll(np.array(T_for_hist), -value)[::cycle]
    J_err = np.roll(np.array(J_for_hist), -value)[::cycle]

    T_low[value_i] = T_ideal[value_i]-np.percentile(T_err, percentile)
    T_high[value_i] = np.percentile(T_err, 100-percentile)-T_ideal[value_i]

    J_low[value_i] = J_ideal[value_i]-np.percentile(J_err, percentile)
    J_high[value_i] = np.percentile(J_err, 100-percentile)-J_ideal[value_i]

# plt.figure('errors')
# ax=plt.gca()
xerr = np.array([T_low, T_high])
yerr = np.array([J_low, J_high])
ax2.errorbar(T_ideal, J_ideal, xerr=xerr, yerr=yerr,
             c='green', fmt='none', capsize=5)

<ErrorbarContainer object of 3 artists>

<ErrorbarContainer object of 3 artists>

In [15]:
def cov_ellipse(cov, q=None, nsig=None, **kwargs):
    """
    from https://stackoverflow.com/a/39749274

    Parameters
    ----------
    cov : (2, 2) array
        Covariance matrix.
    q : float, optional
        Confidence level, should be in (0, 1)
    nsig : int, optional
        Confidence level in unit of standard deviations. 
        E.g. 1 stands for 68.3% and 2 stands for 95.4%.

    Returns
    -------
    width, height, rotation :
         The lengths of two axises and the rotation angle in degree
    for the ellipse.
    """

    if q is not None:
        q = np.asarray(q)
    elif nsig is not None:
        q = 2 * stats.norm.cdf(nsig) - 1
    else:
        raise ValueError('One of `q` and `nsig` should be specified.')
    r2 = stats.chi2.ppf(q, 2)

    val, vec = np.linalg.eigh(cov)
    width, height = 2 * np.sqrt(val[:, None] * r2)
    rotation = np.degrees(np.arctan2(*vec[::-1, 0]))

    return width, height, rotation

In [16]:
'''
#plt.scatter(np.array(T_for_hist),np.array(J_for_hist),s=100)
ax=plt.gca()

#ax.scatter(T,J,color='orange')


for distance_i,distance in enumerate(distances):
    cycle=len(counts)
    value=distance_i
    T_err=np.roll(T_for_hist,value)[::cycle]
    J_err=np.roll(J_for_hist,value)[::cycle]
    width,height,theta=cov_ellipse(np.cov(T_err,J_err),nsig=2)
    ellip = patches.Ellipse(xy=(np.mean(T_err),np.mean(J_err)), width=width, height=height, angle=theta,alpha=0.2,color='black')
    ax.add_artist(ellip)
'''

"\n#plt.scatter(np.array(T_for_hist),np.array(J_for_hist),s=100)\nax=plt.gca()\n\n#ax.scatter(T,J,color='orange')\n\n\nfor distance_i,distance in enumerate(distances):\n    cycle=len(counts)\n    value=distance_i\n    T_err=np.roll(T_for_hist,value)[::cycle]\n    J_err=np.roll(J_for_hist,value)[::cycle]\n    width,height,theta=cov_ellipse(np.cov(T_err,J_err),nsig=2)\n    ellip = patches.Ellipse(xy=(np.mean(T_err),np.mean(J_err)), width=width, height=height, angle=theta,alpha=0.2,color='black')\n    ax.add_artist(ellip)\n"

In [17]:
directories_with_inputs = ['/Volumes/LaCie Drive/Lab/Results/201104/focused_50mm/water','/Volumes/LaCie Drive/Lab/Results/201104/focused_50mm/ice']
states = ['water','ice']
colors = ['orange', 'blue']

file_numbers = [[],[]]

for directory_with_input_index, directory_with_input in enumerate(directories_with_inputs):
    list_of_files = [my_file for my_file in os.listdir(
        directory_with_input) if os.path.isfile(os.path.join(directory_with_input, my_file)) and not my_file.startswith('.')]  # (os.path.isfile(my_file) and (not my_file.startswith('.')))]  # to avoid hidden files like .DS_file
    print(directory_with_input+':')
    print('found {0} files'.format(str(len(list_of_files))))
    list_of_files.sort()
    print('sorted')

    list_of_files_to_do = list_of_files.copy()  # [0:4000:300]
    # print(list_of_files_to_do)
    for file_index, file_name in enumerate(list_of_files_to_do):
        splitted = re.split('_', file_name)
        #print(splitted)
        splitted1 = splitted[-1].split(sep='-')[-1]
        #print(splitted1)
        splitted2 = splitted1.split(sep='.')[-2]
        #print(splitted2)
        file_numbers[directory_with_input_index].append(float(splitted2))





/Volumes/LaCie Drive/Lab/Results/201104/focused_50mm/water:
found 242 files
sorted
/Volumes/LaCie Drive/Lab/Results/201104/focused_50mm/ice:
found 49 files
sorted


In [18]:
bins=np.linspace(0,2000,num=(2+1))
for state_i,state in enumerate(states):
    plt.hist(file_numbers[state_i],bins=bins,label=state,alpha=0.5)
plt.legend()

In [19]:
n,bins,_=plt.hist(file_numbers,bins=bins,label=states,alpha=0.5,stacked=False,color=colors) #to get proper counts
plt.legend()

In [20]:
np.divide(n[1],n[0])

array([0.31578947, 0.1015625 ])

In [21]:
np.divide(n[0],n[1])

array([3.16666667, 9.84615385])

In [22]:
n[:,:1]=np.divide(n[:,:1],2)

In [23]:
n

array([[ 57., 128.],
       [ 18.,  13.]])

In [24]:
62+22

84

In [25]:
13/141

0.09219858156028368